```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/mask.ipynb>`, or a {download}`python script <converted/mask.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial.
```

# Tutorial 3: Masking / Segmentation

```{warning}
This tutorial is currently under development. We will add more information soon.
```

This tutorial explains how to select a region of interest in a video using ``optimap``. For instance, it is possible to use ``optimap`` to automatically select the heart surface, to manually draw a region of interest in the video image, or to ignore parts of the video close to the boundary. The post-processing can then be applied to  the masked or segmented part of the video image, for instance. ``optimap`` provides several easy-to-use routines for these purposes.

We will first load an example video file using the following code (done automatically):

In [ ]:
import optimap as om
import numpy as np

filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um_warped.npy')
video_warped = om.load_video(filepath, use_mmap=True)
om.print_properties(video_warped)

## Automatic Background Segmentation

We can then use ``optimap``'s  {func}`background_mask` function to automatically separate much brighter tissue from the dark background:

In [ ]:
background_mask = om.background_mask(video_warped[0], title='Automatic background mask')
om.save_mask(background_mask, 'background_mask.png')

The {func}`background_mask` function creates a two-dimensional binary array with ``True`` for background (here shown in red) and ``False`` for tissue. Here, we used the first frame of the video to create the mask. {func}`background_mask` and {func}`foreground_mask` automatically estimate an optimal threshold for the separation of foreground and background using the GHT {cite}`Barron2020` algorithm. Pixels with a value below this threshold (here ``401.0``) are considered background, and pixels with a value above this threshold are considered foreground. The threshold can be adjusted manually, if desired:

In [ ]:
foreground_mask = om.foreground_mask(video_warped[0], threshold=500, title='Manual threshold foreground mask')

Here we specified a threshold value of ``500`` to generate a foreground mask. Note that both functions only separate foreground from background, i.e. they do not distinguish the ventricles from the atria. For this purpose, we need to use a different approach.

## Manual Segmentation using Drawing Tool

We can manually draw a mask and select a region (e.g. the atria) using ``optimap``'s mask drawing tool:

In [ ]:
manual_mask = om.interactive_mask(video_warped[0], initial_mask=background_mask)

The drawing tool can be used to draw one or several arbitrary regions and the tool automatically creates the corresponding binary array as a mask. A drawn mask can be inverted or deleted, edits can be undone or redone, etc. (see Documentation). Simply use the above function and a window will pop up:

<img align="center" width="400" src="https://cardiacvision.ucsf.edu/sites/g/files/tkssra6821/f/optimap%20tutorials%20mask%201.png">

<img align="center" width="400" src="https://cardiacvision.ucsf.edu/sites/g/files/tkssra6821/f/optimap%20tutorials%20mask%202.png">

In [ ]:
fn = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um_warped_manual_mask.npy', silent=True)
manual_mask = om.load_mask(fn)

In [ ]:
om.show_mask(manual_mask, image=video_warped[0], title='Manual mask');

## Combining several Masks

The masks `mask1` and `mask2` from above can be used to blank out parts of a video as follows:

In [ ]:
video_warped[:, manual_mask] = np.nan